<a href="https://colab.research.google.com/github/s103021115/Adversarial-Machine-Learning/blob/master/ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Federated learning base on auction game**

##Class Define

### User class

In [79]:
import math
import copy
class User:
  def __init__(self,data_type = "None",iteration = 0,delay = 0,valuation = [0,0]):
    self.data_type = data_type      # type of user : "task_type"
    self.iteration = iteration # iteration count for model : tao
    self.delay = delay     # maximum delay of user : d
    self.valuation = valuation
  def SetType(self,data_type):
    self.data_type = data_type
  def SetIteration(self,iteration):
    self.iteration = iteration
  def SetDelay(self,delay):
    self.delay = delay
  def SetValuation(self,valuation):
    self.valuation = valuation

  def PrintUser(self):
    print("| type : %s | iteration : %d | delay : %d | valuation = [%f,%f]"%(self.data_type,self.iteration,self.delay,self.valuation[0],self.valuation[1]))

  def FitWorkers(self,workers):
    fit_workers = []
    for j in range(len(workers)):
      if workers[j].FindType(self.data_type) > -1:
         fit_workers.append(copy.deepcopy(workers[j]))
    return fit_workers
  def FitWorkersList(self,workers):
    fit_workers_list = []
    for j in range(len(workers)):
      if workers[j].FindType(self.data_type) > -1:
         fit_workers_list.append(j)
    return fit_workers_list
  def Gain(self,data):
    gain = self.valuation[0]*math.log(1+self.valuation[1]*data)
    return gain


####\__init__(data_type, iteration, delay, valuation)
data_type : user data type, string<br>
iteration : user training model iteration<br>
delay   : user maximun tolerate delay time<br>
valuation : [$a_1, a_2$] for valuation function $a_1\ln(a_2D)$, where $D$ is data size<br>

####SetType(data_type)
set user data type

####SetIteration(iteration)
set user train model iteration

####SetDelay(delay)
set user maximum tolerate delay time

####SetValuation(valuation)
set user valuation function

####PrintUser()
print user information

####FitWorkers(workers)
return fit workers.

####FitWorkersList(workers)
return fit workers order

####Gain(data)
return user gain by given data size<br>
$$\alpha\ln(1+\gamma q)$$

### Worker class



In [3]:
class Worker:
  def __init__(self,type_list = []):
    self.type_list = type_list      # type list of worker : [type,data size,type computing cost parameter,type computing time parameter]
  def SetTypeList(self,type_list):
    self.type_list = type_list
  def AddType(self,new_type):
    self.type_list.append(new_type)

  def PrintWorker(self):
    type_number = len(self.type_list)
    print("| type number = %d"%(type_number))
    print("|  type|  size|  cost|  time|")
    for i in range(type_number):
      print("|%10s|%10d|%e|%e|"%(self.type_list[i][0],self.type_list[i][1],self.type_list[i][2],self.type_list[i][3]))

  def FindType(self,data_type): # if exist return number, otherwise return -1
    for i in range(len(self.type_list)): 
      if self.type_list[i][0] == data_type:
        return i
    return -1

  def Cost(self,data_type,size,iteration):
    type_number = self.FindType(data_type)
    if type_number == -1:
      return 0
    else:
      return self.type_list[type_number][2]*size*iteration

  def Delay(self,data_type,size,iteration):
    type_number = self.FindType(data_type)
    if type_number == -1:
      return 0
    else:
      return self.type_list[type_number][3]*size*iteration


####\__init__(type_list)
type list of worker.<br>
type list : [[type 1 information], ... ... ]<br>
type information : [data type, data size, computing cost parament, computing time parament]

####SetTypeList(type_list)
set worker type list.

####AddType(new_type)
add a new type data information to worker.

####PrintWorker()
print worker information.

####FindType(data_type)
return index of data type.<br>
return -1 is do not exist.

####Cost(data_type,size,iteration)
return computing cost.<br>
return 0 is this data type do not exist<br>
$$cqI$$
c : cost parament<br>
q : data size<br>
T : iteration<br>

####Delay(data_type,size,iteration)
return computing dalay time.<br>
return 0 is this data type do not exist<br>
$$dqI$$
d : delay parament<br>
q : data size<br>
T : iteration<br>

### Test

In [4]:
user0 = User("video",10000,10,[9,10])
user0.PrintUser()
print("user gain : %d"%(user0.Gain(10000000000)))
worker0 = Worker([["video",10000000000,0.000000000002,0.000000000004]])
worker0.PrintWorker()
print("worker cost : %f"%(worker0.Cost("video",10000000000,10000)))
print("worker delay : %f"%(worker0.Delay("video",10000000000,10000)))

| type : video | iteration : 10000 | delay : 10 | valuation = [9.000000,10.000000]
user gain : 227
| type number = 1
|  type|  size|  cost|  time|
|     video|10000000000|2.000000e-12|4.000000e-12|
worker cost : 200.000000
worker delay : 400.000000


##Auction Case

In [114]:
def Case_1():
  user0 = User("video",10000,10,[50.0000,10.0000])
  user1 = User("video",10000,10,[40.0000,10.0000])
  user2 = User("video",10000,10,[30.0000,10.0000])
  user3 = User("video",10000,10,[20.0000,10.0000])
  users = [user0,user1,user2,user3]
  worker0 = Worker([["video",100000,0.0000007,0.0000006],["picture",10000,0.0000005,0.0000006],["audio",10000,0.0000005,0.0000006]])
  worker1 = Worker([["video",100000,0.0000007,0.0000006],["picture",10000,0.0000005,0.0000006]])
  worker2 = Worker([["audio",100000,0.0000005,0.0000006]])
  workers = [worker0,worker1,worker2]
  return [users,workers]
def Case_2():
  user0 = User("video",10000,10,[50.0000,10.0000])
  user1 = User("video",10000,10,[40.0000,10.0000])
  user2 = User("video",10000,10,[30.0000,10.0000])
  user3 = User("picture",10000,10,[20.0000,10.0000])
  user4 = User("picture",10000,10,[50.0000,10.0000])
  user5 = User("video",10000,10,[40.0000,10.0000])
  user6 = User("audio",10000,10,[30.0000,10.0000])
  user7 = User("audio",10000,10,[20.0000,10.0000])
  users = [user0,user1,user2,user3,user4,user5,user6,user7]
  worker0 = Worker([["video",100000,0.0000007,0.0000006],["picture",10000,0.0000005,0.0000006],["audio",10000,0.0000005,0.0000006]])
  worker1 = Worker([["video",100000,0.0000007,0.0000006],["picture",10000,0.0000005,0.0000006]])
  worker2 = Worker([["audio",100000,0.0000005,0.0000006]])
  worker3 = Worker([["video",100000,0.0000007,0.0000006],["picture",10000,0.0000005,0.0000006],["audio",10000,0.0000005,0.0000006]])
  worker4 = Worker([["video",100000,0.0000007,0.0000006],["picture",10000,0.0000005,0.0000006]])
  worker5 = Worker([["audio",100000,0.0000005,0.0000006]])
  workers = [worker0,worker1,worker2,worker3,worker4,worker5]
  return [users,workers]

## Server system

###GetAuctionFromJson(filename)
set users and workers by json file.

In [6]:
import json
def GetAuctionFromJson(filename):
  with open(filename) as f:
    data = json.load(f)
  user_number = len(data['users'])
  worker_number = len(data['workers'])
  users = []
  workers = []
  for i in range(user_number):
    new_user = User(data['users'][i]['data_type'], 
             data['users'][i]['iteration'], 
             data['users'][i]['delay'], 
             [data['users'][i]['valuation_a'], data['users'][i]['valuation_b']])
    users.append(new_user)
  for j in range(worker_number):
    type_list = []
    type_number = len(data['workers'][j]['type_list'])
    for k in range(type_number):
      new_type = [data['workers'][j]['type_list'][k]["data_type"],
             data['workers'][j]['type_list'][k]["data_size"],
             data['workers'][j]['type_list'][k]["cost"],
             data['workers'][j]['type_list'][k]["delay"]]
      type_list.append(new_type)
    new_worker = Worker(type_list)
    workers.append(new_worker)
  return[users,workers]

###PrintAuction( users, workers )
print users and workers information.

In [7]:
def PrintAuction(users,workers):
  print("Users")
  print("--------------------------")
  for i in range(len(users)):
    users[i].PrintUser()
    print("--------------------------")
  print("Workers")
  print("--------------------------")
  for i in range(len(workers)):
    workers[i].PrintWorker()
    print("--------------------------")

###UtilityFromUser(user,workers,allocated_data)
return utility from user $i$.<br><br>
Utility Function : 
$$\alpha^i\ln\left(1+\gamma^i\left(\sum_{j\in W}{q^i_j}\right)\right)-\sum_{j\in W}c^j_i\left({q^i_j}\right)$$

In [8]:
def UtilityFromUser(user,workers,allocated_data):
  user_data = 0
  workers_cost = 0
  for j in range(len(workers)):
    user_data += allocated_data[j]
    workers_cost += workers[j].Cost(user.data_type,allocated_data[j],user.iteration)
  user_gain = user.Gain(user_data)
  return (user_gain - workers_cost)

###SocialWelfare(users,workers,allocated_data_list)
return social welfare.<br>
allocation data to user   : n matrix 
allocation data from worker : n X m matrix<br><br>
Objective function : 
$$\sum_{i \in U}{\left(g^i\left(\sum_{j\in W}{q^i_j}\right)-\sum_{j\in W}{c_j\left(q^i_j\right)} \right)}$$

In [9]:
def SocialWelfare(users,workers,allocated_data_list):
  # allocated data to users   : n  matrix
  # allocated data from workers : n*m matrix
  social_welfare = 0
  for i in range(len(users)):
    social_welfare += UtilityFromUser(users[i],workers,allocated_data_list[i])
  return social_welfare

###Constraint(users,workers,allocated_data_list,MAX_DELAY)

In [10]:
def Constraint(users,workers,allocated_data_list,MAX_DELAY):
  time_out = []
  for j in range(len(workers)):
    computing_time = 0
    for i in range(len(users)):
      computing_time += workers[j].Delay(users[i].data_type,allocated_data_list[i][j],users[i].iteration)
    computing_time -= MAX_DELAY
    time_out.append(computing_time)
  return time_out

###Penalty(users,workers,allocated_data_list,MAX_DELAY,mu)
return $||constraint + \mu||^2_2$

In [78]:
def Penalty(users,workers,allocated_data_list,MAX_DELAY,mu):
  panalty_value = 0
  constraint_value = Constraint(users,workers,allocated_data_list,MAX_DELAY)
  for j in range(len(workers)):
    panalty_value += (constraint_value[j]+mu[j])*(constraint_value[j]+mu[j])
    # if (constraint_value[j]>=0):
    #   panalty_value += (constraint_value[j]+mu[j])*(constraint_value[j]+mu[j])
    # else :
    #   panalty_value += mu[j]*mu[j]
  return panalty_value

###AugmentLagarangeForUser(users, workers, allocated_data_list, MAX_DELAY, mu, rho, user_id, user_data)

In [40]:
def AugmentLagarageForUser(users,workers,allocated_data_list,MAX_DELAY,mu,rho,user_id,user_data):
  fit_workerrs_list = users[user_id].FitWorkersList(workers)
  for j in range(len(workers)):
    allocated_data_list[user_id][j] = 0
  for j in range(len(fit_workerrs_list)):
    allocated_data_list[user_id][fit_workerrs_list[j]] = user_data[j]
  return -UtilityFromUser(users[user_id], workers, allocated_data_list[user_id]) + rho/2*Penalty(users,workers,allocated_data_list,MAX_DELAY,mu)

###UpdateUser(users, workers, allocated_data_list, MAX_DELAY, mu, rho)

In [131]:
def UpdateUser(users,workers,allocated_data_list,MAX_DELAY,mu,rho):
  for i in range(len(users)):
    q_bounds = []
    q_i = []
    fit_workerrs_list = users[i].FitWorkersList(workers)
    for j in range(len(fit_workerrs_list)):
      q_bounds.append((0,workers[fit_workerrs_list[j]].type_list[workers[fit_workerrs_list[j]].FindType(users[1].data_type)][1]))
      q_i.append(allocated_data_list[i][fit_workerrs_list[j]])
    arg_q_i = opt.fmin_l_bfgs_b(lambda q: AugmentLagarageForUser(users,workers,allocated_data_list,MAX_DELAY,mu,rho,i,q), q_i, bounds = q_bounds, approx_grad=True)[0]
    for j in range(len(workers)):
      allocated_data_list[i][j] = 0
    for j in range(len(fit_workerrs_list)):
      allocated_data_list[i][fit_workerrs_list[j]] = arg_q_i[j]
  update_mu = Constraint(users,workers,allocated_data_list,MAX_DELAY)
  for j in range(len(workers)):
    mu[j] = mu[j] + update_mu[j]
  return [allocated_data_list,mu]

###TwoNormError(a,b)

In [96]:
def TwoNormError(a,b):
  differnt = 0
  for i in range(len(a)):
    differnt += (a[i]-b[i])*(a[i]-b[i])
  return differnt

###ADMM(users, workers, allocated_data_list, MAX_DELAY, error, MAX_count)

In [133]:
import copy
import math
def ADMM(users,workers,allocated_data_list,MAX_DELAY,error,MAX_count):
  mu =[]
  rho = 1
  for j in range(len(workers)):
    mu.append(1)
  prev_mu = copy.deepcopy(mu)
  [allocated_data_list,mu] = UpdateUser(users,workers,allocated_data_list,MAX_DELAY,mu,rho)
  count = 0
  while TwoNormError(prev_mu,mu) >= error and count <= MAX_count:
    prev_mu = copy.deepcopy(mu)
    [allocated_data_list,mu] = UpdateUser(users,workers,allocated_data_list,MAX_DELAY,mu,rho)
    count += 1
  prev_mu = copy.deepcopy(mu)
  [allocated_data_list,mu] = UpdateUser(users,workers,allocated_data_list,MAX_DELAY,mu,rho)
  return allocated_data_list

##Test

In [139]:
# [users,workers] = GetAuctionFromJson("data.json")
def test_1():
  print("Test case 1.")
  [users,workers] = Case_1()
  allocated_data_list = [[6000,5000,0],[6000,4000,0],[6000,3000,0],[6000,2000,0]]
  mu = [5,5,5]
  prev_mu = mu
  rho = 2
  MAX_DELAY = 200
  allocated_data_list = ADMM(users,workers,allocated_data_list,MAX_DELAY,1.0e-06,500)
  print(allocated_data_list)
  print(SocialWelfare(users,workers,allocated_data_list))
  print("-----------------------------------------------")

def test_2():
  print("Test case 2.")
  [users2,workers2] = Case_2()
  PrintAuction(users2,workers2)
  allocated_data_list2 = [[6000,5000,0,0,0,0],[6000,4000,0,0,0,0],[6000,3000,0,0,0,0],[6000,2000,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[8000,10000,0,0,0,0]]
  mu = [5,5,5]
  prev_mu = mu
  rho = 2
  MAX_DELAY = 200
  print(SocialWelfare(users2,workers2,allocated_data_list2))
  allocated_data_list2 = ADMM(users2,workers2,allocated_data_list2,MAX_DELAY,1.0e-06,500)
  print("Allocated data list : ")
  for i in range(len(users2)):
    print("user %d"%(i))
    print(allocated_data_list2[i])
  constraint_for_worker2 = Constraint(users2,workers2,allocated_data_list2,MAX_DELAY)
  print("Constraint : ")
  for j in range(len(workers2)):
    print("worker %d"%(j))
    print(constraint_for_worker2[j])
  print("SocialWelfare : ")
  print(SocialWelfare(users2,workers2,allocated_data_list2))
  print("-----------------------------------------------")

# test_1()
test_2()

Test case 2.
Users
--------------------------
| type : video | iteration : 10000 | delay : 10 | valuation = [50.000000,10.000000]
--------------------------
| type : video | iteration : 10000 | delay : 10 | valuation = [40.000000,10.000000]
--------------------------
| type : video | iteration : 10000 | delay : 10 | valuation = [30.000000,10.000000]
--------------------------
| type : picture | iteration : 10000 | delay : 10 | valuation = [20.000000,10.000000]
--------------------------
| type : picture | iteration : 10000 | delay : 10 | valuation = [50.000000,10.000000]
--------------------------
| type : video | iteration : 10000 | delay : 10 | valuation = [40.000000,10.000000]
--------------------------
| type : audio | iteration : 10000 | delay : 10 | valuation = [30.000000,10.000000]
--------------------------
| type : audio | iteration : 10000 | delay : 10 | valuation = [20.000000,10.000000]
--------------------------
Workers
--------------------------
| type number = 3
|  type| 

### Ver.3
<dl>
    <dd>利用 ADMM 來找最佳解</dd>
    <dd>利用 VCG 算 payment</dd>
</dl>
<dl>
<dt>步驟:</dt>
  <dd>ADMM Method:<br>
    Objective Function:
  </dd>
  $$\max_{X}{\sum_{i \in U}{\left(v^i\left(\sum_{j \in W}x^{i}_{j}\right) -\sum_{j \in W}{c_j(x^i_j)}\right)}}$$
</dl>

#ADMM



## CASE 1
Objective Function :
$$f(x)=(x_1)^2+(x_2)^2+(x_3)^2$$
subject to :
$$x_1+x_2+x_3=1$$
Lagrangian Multiplier:
$$(x_1)^2+(x_2)^2+(x_3)^2 + y(x_1+x_2+x_3-1)$$
ADMM:
$$L_p(x_1,x_2,x_3,y)=(x_1)^2+(x_2)^2+(x_3)^2+\frac{\rho}{2}||x_1+x_2+x_3-1+y||_2^2$$
update:<br>
$x_i = \arg\min x_i^2 + \frac{\rho}{2}||x_1+x_2+x_3-1+y||$<br>
$y = y + x_1+x_2+x_3-1$

In [ ]:
import scipy.optimize as opt
import scipy
from numpy import *
def object(x):
  return (x[0]**(2)+x[1]**(2)+x[2]**(2))

def constraint(x):
  return x[0]+x[1]+x[2]-1

def lagrangian(x,l,r = 1):
  return object(x) + r*(constraint(x)+l)**(2)

def setL(x,l):
  l = l + constraint(x)
  return l

def setValue(arg_x,arg_l,r = 1):
  arg_x = opt.fmin(lambda x: lagrangian(x,arg_l,r), [0,0,0], disp = 0)
  arg_l = arg_l + constraint(arg_x)
  return [arg_x,arg_l]

prev_value = [[5,5,5],5]
prev = lagrangian(prev_value[0],prev_value[1])
now_value = setValue(prev_value[0],prev_value[1])
now = lagrangian(now_value[0],now_value[1])
count = 0
while abs(prev_value[1] - now_value[1]) > 1.0e-06 and count < 10000:
  prev_value = now_value
  now_value = setValue(prev_value[0],prev_value[1],1)
  prev = now
  now = lagrangian(now_value[0],now_value[1])
  count += 1
  # print("turn %d"%(count))
  # print("  x = [%e,%e,%e]"%(now_value[0][0],now_value[0][1],now_value[0][2]))
  # print("  y = %e"%(now_value[1]))
ans = opt.fmin(lambda x: lagrangian(x,now_value[1]), [0,0,0], disp = 0)
print(now_value)
print(now)
print(ans)
print(object(ans))
print(constraint(ans))

[array([0.33331224, 0.33333501, 0.33335262]), -0.33330953784043926]
0.44442858216226633
[0.33331224 0.33333501 0.33335262]
0.3333332460116796
-1.32212181203073e-07


## CASE 2
objective function:
$$f(x) = 6\ln(5x_1)+4\ln(7x_2)+2\ln(9x_3) - (0.001x_1 + 0.002x_2 + 0.003x_3)$$
subject to:
$$0.0004x_1 + 0.0004x_2 + 0.0004x_3\le500$$
lagrangian:
$$6\ln(5x_1)+4\ln(7x_2)+2\ln(9x_3) - (0.001x_1 + 0.002x_2 + 0.003x_3) + \frac{\rho}{2}||0.0004x_1 + 0.0004x_2 + 0.0004x_3 - 500 +y||^2_2$$

In [23]:
import scipy.optimize as opt
import scipy
from numpy import *
import math
def object2(x):
  return -(6*math.log(1+5*x[0])+4*math.log(1+7*x[0])+2*math.log(1+9*x[0])-0.001*x[0]-0.002*x[0]-0.003*x[0] )

def constraint2(x):
  return 0.00004*x[0]+0.00004*x[1]+0.0004*x[2]-500

def lagrangian2(x,l,r=1):
  return object2(x) + r*(constraint2(x)+l)**(2)

def dual_object2(l):
  x_bounds = [(0,100000000000),(0,100000000000),(0,100000000000)]
  arg_x = opt.fmin_l_bfgs_b(lambda x: lagrangian2(x,l), [5,5,5], bounds = x_bounds ,approx_grad=True)[0]
  return object2(arg_x)

def setValue2(arg_x,arg_l,r=1):
  x_bounds = [(0,100000000000),(0,100000000000),(0,100000000000)]
  arg_x = opt.fmin_l_bfgs_b(lambda x: lagrangian2(x,arg_l,r), arg_x, bounds = x_bounds,approx_grad=True)[0]
  arg_l = arg_l + constraint2(arg_x)
  return [arg_x,arg_l]

prev_value2 = [[5,5,5],5]
now_value2 = setValue2(prev_value2[0],prev_value2[1])
prev2 = lagrangian2(prev_value2[0],prev_value2[1])
now2 = lagrangian2(now_value2[0],now_value2[1])
count2 = 0
a2 = 0.25
while abs(now_value2[1] - prev_value2[1]) > 1.0e-05 and count2 < 500:
  prev_value2 = now_value2
  now_value2 = setValue2(prev_value2[0],prev_value2[1],a2)
  prev2 = now2
  now2 = lagrangian2(now_value2[0],now_value2[1])
  count2 += 1
  # print("turn %d"%(count2))
  # print("  x = [%e,%e,%e]"%(now_value2[0][0],now_value2[0][1],now_value2[0][2]))
  # print("  y = %e"%(now_value2[1]))
  # print("------------------------------")
  # print(count)
  # a = a * 0.5
ans2 = opt.fmin_l_bfgs_b(lambda x: lagrangian2(x,now_value2[1]), now_value2[0],bounds = [(0,100000000000),(0,100000000000),(0,100000000000)],approx_grad=True)[0]
print(now_value2)
print(now2)
print(ans2)
print(object2(ans2))
print(constraint2(ans2))
test_l_ans = opt.fmin_l_bfgs_b(lambda x: -dual_object2(x), 5,approx_grad=True)[0]
test_x_ans = opt.fmin_l_bfgs_b(lambda x: lagrangian2(x,test_l_ans), [5,5,5],bounds = [(0,100000000000),(0,100000000000),(0,100000000000)],approx_grad=True)[0]
print("other way")
print(test_l_ans)
print("%e"%(test_x_ans[0]))
print("%e"%(test_x_ans[1]))
print("%e"%(test_x_ans[2]))
print(object2(test_x_ans))
print(constraint2(test_x_ans))

[array([   2000.48181503,  304840.39455471, 1219311.66093678]), -0.8563017918078231]
-100.31037488016084
[   1998.83657534  305256.24463581 1221415.40031068]
-101.04654207114179
0.856363372716487
other way
[5.00980474]
1.997198e+03
5.033970e+04
1.232247e+06
-101.04653313661849
-5.007570410213134


##CASE 3
objective :
$$f(x_1,x_2,x_3) = 9\ln(5(x_1+x_2+x_3))-0.00001x_1-0.00001x_2-0.00001x_3$$
constraint : 
$$x_1+x_2+x_3 \le 50000$$
largangiant:
$$\ln(x_1+x_2)+\ln(x_3)-x_1-x_2-x_3 + \frac{\rho}{2}||x_1+x_2+x_3-500+y||_2^2$$

In [ ]:
import scipy.optimize as opt
import scipy
from numpy import *
import math
def object3(x):
  return -(9*math.log(5*(x[0]+x[1]+x[2]))-0.00001*(x[0]+x[1]+x[2]))
def constraint3(x):
  return x[0]+x[1]+x[2]-50000
def lagrangian3(x,y,r=1):
  return object3(x)+r*(constraint3(x)+y)**(2)
def setValue3(arg_x,arg_y,r=1):
  arg_x = opt.fmin_l_bfgs_b(lambda x: lagrangian3(x,arg_y,r), arg_x, bounds = [(0,100000000000),(0,100000000000),(0,100000000000)],approx_grad=True)[0]
  arg_y = arg_y + constraint3(arg_x)
  return [arg_x,arg_y]

prev_value3 = [[5,5,5],5]
now_value3 = setValue3(prev_value3[0],prev_value3[1])
count3 = 0
while abs(prev_value3[1]-now_value3[1]) > 1.0e-06 and count3 < 50000:
  prev_value3 = now_value3
  now_value3 = setValue3(prev_value3[0],prev_value3[1])
  count3 += 1
  print("turn %d"%(count3))
  print("  x = [%e,%e,%e]"%(now_value3[0][0],now_value3[0][1],now_value3[0][2]))
  print("  y = %e"%(now_value3[1]))
  print("------------------------------")


print(object3(now_value3[0]))
print(constraint3(now_value3[0]))

turn 1
  x = [1.666666e+04,1.666666e+04,1.666667e+04]
  y = 8.789101e-05
------------------------------
turn 2
  x = [1.666666e+04,1.666666e+04,1.666667e+04]
  y = 9.057818e-05
------------------------------
turn 3
  x = [1.666666e+04,1.666666e+04,1.666667e+04]
  y = 8.517841e-05
------------------------------
turn 4
  x = [1.666666e+04,1.666666e+04,1.666667e+04]
  y = 8.509734e-05
------------------------------
-111.36294577158567
-8.106871973723173e-08


test

In [ ]:
import scipy.optimize as opt
import scipy
from numpy import *
import math
def f(x):
  print(x)
  return (6*math.log(1+5*x[0])+4*math.log(1+7*x[1])+2*math.log(1+9*x[2]))

# arg = opt.fmin_l_bfgs_b(lambda x : f(x),[100,100,100])
arg = opt.fmin_l_bfgs_b(lambda x : f(x),[100,100,100],bounds = [(0,100000),(0,100000),(0,100000)],approx_grad=True)
print(arg[0])


In [ ]:
import scipy.optimize as opt
import scipy
from numpy import *
import math
def f(x):
    print(x)
    return math.log(x[0]**(2)) + x[1]

max_x = opt.fmin_l_bfgs_b(lambda x: f(x), [1.0,1.0], bounds=[(-9,9),(-9,9)],approx_grad=True)
print(max_x)